In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
from matplotlib import pyplot as plt

%matplotlib inline
plt.rcParams["font.family"] = "Malgun Gothic"
plt.rcParams["font.size"] = 15
plt.rcParams["figure.figsize"] = (16,9)
plt.rcParams["axes.unicode_minus"] = False

# 0.Data Load
---

In [3]:
df_pdde = pd.read_csv("../LPOINT_BIG_COMP/LPOINT_BIG_COMP_02_PDDE.csv", low_memory=False)
df_copu = pd.read_csv("../LPOINT_BIG_COMP/LPOINT_BIG_COMP_03_COP_U.csv")
df_lpay = pd.read_csv("../LPOINT_BIG_COMP/LPOINT_BIG_COMP_06_LPAY.csv")

In [4]:
df_pdde.head()

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1


In [5]:
df_copu.head()

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026,12,60000
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106,10,17100
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211,12,136500
3,M839993508,21092010B012637545,B01,B010012,1,20210920,20210920,10,34200
4,M839993508,21101009D015920171,D01,D010614,1,20211010,20211010,9,2500


In [6]:
df_lpay.head()

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
0,M629656521,210803210311226,A03,1,20210803,21,10900
1,M216016456,210803130167542,L01,2,20210803,13,6860
2,M205142844,210803140275112,A02,1,20210803,14,9000
3,M737010483,210803040637594,A06,2,20210803,4,36740
4,M707775545,210803140675502,A06,2,20210803,14,138500


In [7]:
print(f"PDDE\t{len(df_pdde)}")
print(f"COPU\t{len(df_copu)}")
print(f"LPAY\t{len(df_lpay)}")

PDDE	4381743
COPU	248304
LPAY	353184


In [14]:
for col in df_copu.columns:
    print(f"{col}\t{df_copu[col].nunique()}")

cust	21769
rct_no	248304
cop_c	6
br_c	2029
chnl_dv	2
de_dt	369
vst_dt	366
de_hr	24
buy_am	9420


In [15]:
df_copu.head()

,cust,rct_no,cop_c,br_c,chnl_dv,de_dt,vst_dt,de_hr,buy_am
0,M839993508,21102612B015763935,B01,B010012,1,20211026,20211026,12,60000
1,M839993508,21110610B014219744,B01,B010012,1,20211106,20211106,10,17100
2,M839993508,21021112B013419710,B01,B010012,1,20210211,20210211,12,136500
3,M839993508,21092010B012637545,B01,B010012,1,20210920,20210920,10,34200
4,M839993508,21101009D015920171,D01,D010614,1,20211010,20211010,9,2500


- `PDDE`와 `COP_U` 경우 유통사 제품 구매와, 제휴사 서비스 이용 내역이기 때문에 독립적인 테이블임
- `LPAY`는 엘페이를 이용한 일종의 결제 내역이기 때문에 `PDDE`와 `COP_U` 테이블과의 연관성이 있을 것으로 보임
- 따라서 확인 해야할 사항은 다음과 같음
    1. `PDDE`-`LPAY`
    2. `COP_U`-`LPAY`
- 추가적으로 통일되어 있지 않은 영수증 번호에 대한 고찰도 수행

# 1.Transaction Data 간 관계 파악
---

## 1.1. `PDDE` - `LPAY`

In [85]:
df_lpay["rct_no"]=df_lpay["rct_no"].astype("str")

In [86]:
df_pd = pd.read_csv("../LPOINT_BIG_COMP/LPOINT_BIG_COMP_04_PD_CLAC.csv")
df_pd.head()

,pd_c,pd_nm,clac_hlv_nm,clac_mcls_nm
0,PD0001,소파,가구,거실가구
1,PD0002,스툴/리빙의자,가구,거실가구
2,PD0003,탁자,가구,거실가구
3,PD0004,장식장/진열장,가구,거실가구
4,PD0005,기타가구,가구,기타가구


In [104]:
df_pdmr = df_pdde.merge(df_pd[["pd_c","pd_nm"]], on="pd_c", how="left")
df_pdmr

,cust,rct_no,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,남성티셔츠
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,기타주방일회용품
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠
...,...,...,...,...,...,...,...,...,...,...,...
4381738,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권
4381739,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권
4381740,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권
4381741,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권


In [110]:
len(df_pdmr.loc[df_pdmr.duplicated()]["pd_nm"].unique().tolist())

1465

In [88]:
temp=pd.merge(left=df_pdmr, right=df_lpay, on=["cust","chnl_dv","cop_c","de_dt","de_hr","buy_am"], how="left")

In [89]:
temp

,cust,rct_no_x,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,rct_no_y
0,M430112881,A01000001113,1,A01,A010039,PD0290,20210101,10,15000.0,1,남성티셔츠,NaN
1,M646853852,A01000002265,1,A01,A010025,PD1369,20210101,10,79700.0,1,기타주방일회용품,NaN
2,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,NaN
3,M430112881,A01000003148,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,NaN
4,M430112881,A01000004946,1,A01,A010039,PD0290,20210101,10,19000.0,1,남성티셔츠,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
4382534,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권,NaN
4382535,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권,NaN
4382536,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권,NaN
4382537,M533286446,E06052119978,2,A06,NaN,PD0507,20211231,23,50000.0,1,백화점/마트모바일상품권,NaN


In [90]:
temp.loc[~temp["rct_no_y"].isna()]

,cust,rct_no_x,chnl_dv,cop_c,br_c,pd_c,de_dt,de_hr,buy_am,buy_ct,pd_nm,rct_no_y
24,M686795542,A01000022861,1,A01,A010045,PD0952,20210101,11,34800.0,1,여성베스트,210101110137006
46,M511106441,A01000034639,1,A01,A010024,PD0777,20210101,12,59100.0,1,남성런닝/트레이닝화,210101120141530
68,M511106441,A01000047099,1,A01,A010024,PD0777,20210101,13,50400.0,1,남성런닝/트레이닝화,210101130143937
79,M049345995,A01000057303,1,A01,A010043,PD1827,20210101,13,39900.0,2,기타패션잡화,210101130144874
114,M502480776,A01000087195,1,A01,A010045,PD1692,20210101,14,5500.0,1,커피/음료,210101140148885
...,...,...,...,...,...,...,...,...,...,...,...,...
4382429,M324896636,E06052060518,2,A06,NaN,PD0507,20211229,16,50000.0,1,백화점/마트모바일상품권,211229160663040
4382430,M305766997,E06052061811,2,A06,NaN,PD0511,20211229,16,9800.0,1,식음료모바일상품권,211229160665309
4382431,M305766997,E06052061811,2,A06,NaN,PD0511,20211229,16,9800.0,1,식음료모바일상품권,211229160665356
4382432,M305766997,E06052062286,2,A06,NaN,PD0511,20211229,16,9800.0,1,식음료모바일상품권,211229160665309


In [91]:
df_rctcust = df_pdmr.groupby(by=["rct_no","cust","de_dt","de_hr"], as_index=False)["chnl_dv"].count()
df_rctcust

,rct_no,cust,de_dt,de_hr,chnl_dv
0,A01000001113,M430112881,20210101,10,1
1,A01000002265,M646853852,20210101,10,1
2,A01000003148,M430112881,20210101,10,2
3,A01000004946,M430112881,20210101,10,1
4,A01000005297,M430112881,20210101,10,1
...,...,...,...,...,...
1266584,E06052115831,M816318679,20211231,22,1
1266585,E06052116037,M816318679,20211231,22,3
1266586,E06052117103,M182645944,20211231,23,1
1266587,E06052118403,M533286446,20211231,23,5


In [92]:
df_pdlp = df_pdmr.groupby(by=["rct_no","de_dt","de_hr"], as_index = False)["buy_am"].aggregate(["count", "sum"])
temp = pd.merge(left=df_pdlp, right=df_rctcust[["rct_no","cust","de_dt","de_hr"]], on="rct_no", how="left").sort_values(by="count", ascending=False)
temp

,rct_no,count,sum,cust,de_dt,de_hr
591239,A02090597964,109,701460.0,M159274242,20210128,19
735311,A02237644368,93,160160.0,M210088658,20210925,22
846163,A02350722306,91,570710.0,M999515910,20211021,11
520242,A02018116148,82,616032.0,M003531710,20210131,16
699207,A02200575126,82,756930.0,M050101003,20210928,13
...,...,...,...,...,...,...
167920,A01168101496,1,7600.0,M285726188,20210507,11
167919,A01168100004,1,5800000.0,M222501555,20210507,11
167918,A01168099740,1,7100.0,M610673889,20210507,11
167917,A01168098301,1,37050.0,M073553732,20210507,11


In [93]:
# -- 유통사에서 LPAY를 이용하여 구매한 내역
df_pdlp = temp.merge(df_lpay, how="left", left_on=["cust","de_dt","de_hr","sum"], right_on=["cust","de_dt","de_hr","buy_am"])
pdde_rct = df_pdlp.loc[~df_pdlp["buy_am"].isna()]["rct_no_y"].tolist()
df_pdlp.loc[~df_pdlp["buy_am"].isna()]

,rct_no_x,count,sum,cust,de_dt,de_hr,rct_no_y,cop_c,chnl_dv,buy_am
1279,A02344466200,37,289800.0,M008269534,20210716,16,210716160207112,A02,1.0,289800.0
6577,A02301538351,26,116930.0,M616195835,20210502,16,210502160201319,A02,1.0,116930.0
6753,A02188250565,26,96760.0,M626028376,20210508,19,210508190242902,A02,1.0,96760.0
8464,A02038651604,25,137420.0,M347139064,20210810,21,210810210294971,A02,1.0,137420.0
8737,A01163060143,25,99640.0,M805001563,20210504,11,210504110198133,A01,1.0,99640.0
...,...,...,...,...,...,...,...,...,...,...
1267382,A02054328236,1,10000.0,M915239352,20210609,11,210609110275961,A02,1.0,10000.0
1267388,A01168126496,1,59000.0,M969276038,20210507,12,210507120190279,A01,1.0,59000.0
1267416,A01168081805,1,26500.0,M264007732,20210507,11,210507110187477,A01,1.0,26500.0
1267439,A01168101496,1,7600.0,M285726188,20210507,11,210507110189223,A01,1.0,7600.0


- 결제 내역 or 결제 수단에 대해서는 `rct_no`로 group화 해서 결합
- 고객 개인화 전략에 대해서는 품목에 포커스를 두는게 좋음 (품목 누락 없이)

In [94]:
df_copu
df_rctcust = df_copu.groupby(by=["rct_no","cust","de_dt","de_hr"], as_index=False)["chnl_dv"].count()
df_rctcust

,rct_no,cust,de_dt,de_hr,chnl_dv
0,20122713C012328716,M495972460,20201227,13,1
1,20122900C013317930,M192531170,20201229,0,1
2,20122911C011862577,M039320884,20201229,11,1
3,20123009C012921880,M201983289,20201230,9,1
4,20123012C011060420,M205977381,20201230,12,1
...,...,...,...,...,...
248299,21123122E012116298,M261613965,20211231,22,1
248300,21123122E012916542,M716433837,20211231,22,1
248301,21123122E012920649,M236790127,20211231,22,1
248302,21123123D010326849,M449095124,20211231,23,1


In [95]:
df_colp = df_copu.groupby(by=["rct_no","de_dt","de_hr"], as_index = False)["buy_am"].aggregate(["count", "sum"])
temp = pd.merge(left=df_colp, right=df_rctcust[["rct_no","cust","de_dt","de_hr"]], on="rct_no", how="left").sort_values(by="count", ascending=False)
temp

,rct_no,count,sum,cust,de_dt,de_hr
0,20122713C012328716,1,26000,M495972460,20201227,13
165556,21090512D010860495,1,6150,M064660141,20210905,12
165528,21090512C010760248,1,6000,M228508070,20210905,12
165529,21090512C011042119,1,44600,M250355096,20210905,12
165530,21090512C011059330,1,5500,M376415415,20210905,12
...,...,...,...,...,...,...
82772,21051819D014898136,1,9000,M100924636,20210518,19
82773,21051819D015061179,1,6200,M497442092,20210518,19
82774,21051819D015476203,1,7800,M045416783,20210518,19
82775,21051819D015913730,1,17600,M024139053,20210518,19


In [96]:
# -- 유통사에서 LPAY를 이용하여 구매한 내역
df_colp = temp.merge(df_lpay, how="left", left_on=["cust","de_dt","de_hr","sum"], right_on=["cust","de_dt","de_hr","buy_am"])
copu_rct = df_colp.loc[~df_colp["buy_am"].isna()]["rct_no_y"].tolist()
df_colp.loc[~df_colp["buy_am"].isna()]

,rct_no_x,count,sum,cust,de_dt,de_hr,rct_no_y,cop_c,chnl_dv,buy_am
3,21090512C011042119,1,44600,M250355096,20210905,12,210905120191578,C01,2.0,44600.0
9,21090512C012537618,1,34000,M614101488,20210905,12,210905120193237,C01,2.0,34000.0
24,21090512D010195514,1,9300,M247140505,20210905,12,210905120188234,D01,2.0,9300.0
30,21090512C010310545,1,18000,M250355096,20210905,12,210905120192696,C01,2.0,18000.0
38,21090511D013617771,1,105000,M194134921,20210905,11,210905110185879,D01,2.0,105000.0
...,...,...,...,...,...,...,...,...,...,...
248740,21051820C021734793,1,30000,M916971028,20210518,20,210518200268751,C02,2.0,30000.0
248750,21051820D011956085,1,5900,M781690028,20210518,20,210518200168449,D01,1.0,5900.0
248758,21051819C025461511,1,60000,M203953437,20210518,19,210518190261566,C02,2.0,60000.0
248761,21051819D010820118,1,27000,M768349404,20210518,19,210518190166504,D01,2.0,27000.0


In [100]:
df_lpay.loc[~df_lpay["rct_no"].isin(pdde_rct+copu_rct)]

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
1,M216016456,210803130167542,L01,2,20210803,13,6860
3,M737010483,210803040637594,A06,2,20210803,4,36740
4,M707775545,210803140675502,A06,2,20210803,14,138500
5,M848083175,210803180197432,A01,1,20210803,18,14000
9,M161654425,210803190302973,A03,1,20210803,19,19010
...,...,...,...,...,...,...,...
353176,M220008309,210113130538421,A05,2,20210113,13,129000
353177,M920715369,210110150608046,A06,2,20210110,15,24000
353178,M470112503,210104210651243,A06,2,20210104,21,50920
353181,M748878049,210119200278240,A02,1,20210119,20,17600


In [98]:
df_lpay.loc[df_lpay["rct_no"].isin(pdde_rct+copu_rct)]

,cust,rct_no,cop_c,chnl_dv,de_dt,de_hr,buy_am
0,M629656521,210803210311226,A03,1,20210803,21,10900
2,M205142844,210803140275112,A02,1,20210803,14,9000
6,M253450307,210803160182662,C01,2,20210803,16,1000
7,M186539937,210803130266856,D02,1,20210803,13,9600
8,M347475637,210803160182823,C01,2,20210803,16,1000
...,...,...,...,...,...,...,...
353171,M688241278,210123150218189,A02,1,20210123,15,5460
353175,M484453427,210131160335899,A03,1,20210131,16,9900
353179,M470112503,210125110644545,A06,2,20210125,11,51800
353180,M470112503,210104210652271,A06,2,20210104,21,6440


In [102]:
df_pdlp.loc[df_pdlp["cust"]=="M216016456"]

,rct_no_x,count,sum,cust,de_dt,de_hr,rct_no_y,cop_c,chnl_dv,buy_am
41689,A02286222824,15,53670.0,M216016456,20210208,15,NaN,NaN,NaN,NaN
135283,A02287417733,8,30960.0,M216016456,20210731,16,NaN,NaN,NaN,NaN
236554,A02286779729,5,25220.0,M216016456,20210501,17,NaN,NaN,NaN,NaN
274591,A02134818112,5,9260.0,M216016456,20211022,20,NaN,NaN,NaN,NaN
383185,A04041337694,3,10800.0,M216016456,20210523,16,NaN,NaN,NaN,NaN
416167,A02134985259,3,8400.0,M216016456,20211231,18,NaN,NaN,NaN,NaN
548650,A02303743684,2,47600.0,M216016456,20210501,17,NaN,NaN,NaN,NaN
710787,E06035154748,1,6200.0,M216016456,20210502,19,NaN,NaN,NaN,NaN
710804,E06035153273,1,6200.0,M216016456,20210502,19,NaN,NaN,NaN,NaN
717864,E06026425674,1,25900.0,M216016456,20210410,10,NaN,NaN,NaN,NaN


In [103]:
df_colp.loc[df_colp["cust"]=="M216016456"]

,rct_no_x,count,sum,cust,de_dt,de_hr,rct_no_y,cop_c,chnl_dv,buy_am
19582,21100315D014953006,1,5300,M216016456,20211003,15,NaN,NaN,NaN,NaN
102968,21101014D010539137,1,3900,M216016456,20211010,14,NaN,NaN,NaN,NaN
163674,21010716D015810054,1,12000,M216016456,20210107,16,NaN,NaN,NaN,NaN
178209,21022415D010639779,1,3700,M216016456,20210224,15,NaN,NaN,NaN,NaN
178225,21022415D013434767,1,3700,M216016456,20210224,15,NaN,NaN,NaN,NaN
204115,21070414D011613261,1,15600,M216016456,20210704,14,NaN,NaN,NaN,NaN
220927,21050219D011072467,1,7800,M216016456,20210502,19,NaN,NaN,NaN,NaN
224158,21050514D012578653,1,7800,M216016456,20210505,14,NaN,NaN,NaN,NaN
245390,21051916D012320056,1,2000,M216016456,20210519,16,NaN,NaN,NaN,NaN
245409,21051916D011130666,1,3700,M216016456,20210519,16,210519160139071,D01,2.0,3700.0


1. LPAY O / LPAY X 어떻게 구분할 것인가? -> 완벽히 구분한다? 주제 설정이 수월
2. PDDE - 중복이 무엇인가?
3. LPAY가 연결 안 되는 데이터는 무엇인가?